# What are the most demanded skills for the top 3 most popular data roles?

## Methodology
1. Clean-up skill column;
2. Calculate skill count based on job_title_short
3. Pivot initial findings
4. Calculate skill percentage
5. Plot final findings

## Original Exploration
#### Matplotlib Format Charts

### Import Libraries and Data

In [28]:
import ast
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
df = pd.read_csv('/home/renat/Desktop/programming/data_science/youtube/luke_barousse/python-for-data-analytics/data/data_jobs.csv')

In [30]:
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

In [31]:
"""Этот код преобразует значения в столбце job_skills из строкового представления списка в сам список, если значение не является NaN.
pd.notna(x) проверяет, что значение не является NaN.
ast.literal_eval(x) безопасно преобразует строку, содержащую список (например, "['Python', 'Pandas']") в настоящий список ['Python', 'Pandas'].
Если значение NaN, оно остаётся без изменений.
Пример:
Было: "[ 'Python', 'Pandas' ]" → стало: ['Python', 'Pandas']"""

df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

In [32]:
df_US = df[df['job_country'] == 'United States']

In [37]:
df.shape

(785741, 17)

In [33]:
df_skills = df_US.explode('job_skills')

In [36]:
df_skills.shape

(1073565, 17)

In [34]:
df_skills[['job_title', 'job_skills']]

,job_title,job_skills
0,Senior Clinical Data Engineer / Principal Clin...,NaN
3,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,python
3,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,c++
3,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,java
3,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,matlab
...,...,...
785692,Data Scientist- Hybrid Work Location,r
785703,Data Analyst - CRYPTOGRAPHY - Full-time,NaN
785705,Expert Business Data Analyst - Now Hiring,sql
785705,Expert Business Data Analyst - Now Hiring,python


In [38]:
df_skills_count = df_skills.groupby(['job_skills', 'job_title_short']).size()
df_skills_count

job_skills  job_title_short          
airflow     Business Analyst               38
            Cloud Engineer                 27
            Data Analyst                  387
            Data Engineer                4716
            Data Scientist               1166
                                         ... 
zoom        Machine Learning Engineer       5
            Senior Data Analyst            50
            Senior Data Engineer           30
            Senior Data Scientist          25
            Software Engineer              16
Length: 1870, dtype: int64

In [ ]:
# Change df_skills_count from type "Series" to "DataFrame"
df_skills_count = df_skills_count.reset_index(name='skill_count')

In [ ]:
df_skills_count

In [ ]:
df_skills_count.sort_values(by='skill_count', ascending=False, inplace=True)

In [ ]:
df_skills_count

In [ ]:
# Создаем лист
job_titles = df_skills_count['job_title_short'].unique().tolist()
job_titles

In [ ]:
# Берем только первые 3
job_titles = sorted(job_titles[:3])
job_titles

In [ ]:
fig, ax = plt.subplots(len(job_titles), 1)

for i, job_title in enumerate(job_titles):
    df_plot = df_skills_count[df_skills_count['job_title_short'] == job_title].head(5)
    df_plot.plot(kind='barh', x='job_skills', y='skill_count', ax=ax[i], title=job_title)
    ax[i].invert_yaxis()
    ax[i].set_ylabel('')
    ax[i].legend().set_visible(False)

fig.suptitle('Count of Top Skills in Job Postings', fontsize=15)
plt.tight_layout(h_pad=0.5) # firx the overlap
plt.show()

In [ ]:
df_job_title_count = df_US['job_title_short'].value_counts().reset_index(name='jobs_total')
df_job_title_count

In [ ]:
df_skill_perc = pd.merge(df_skills_count, df_job_title_count, how='left', on='job_title_short')
df_skill_perc

In [ ]:
df_skill_perc['skill_percent'] = 100 * df_skill_perc['skill_count'] / df_skill_perc['jobs_total']
df_skill_perc

In [ ]:
fig, ax = plt.subplots(len(job_titles), 1)

sns.set_theme(style='ticks')

for i, job_title in enumerate(job_titles):
    df_plot = df_skill_perc[df_skill_perc['job_title_short'] == job_title].head(5)
    sns.barplot(data=df_plot, x='skill_percent', y='job_skills', ax=ax[i], hue='skill_count', palette='dark:b_r')
    ax[i].set_title(job_title)
    ax[i].set_ylabel('')
    ax[i].set_xlabel('')
    ax[i].get_legend().remove()
    ax[i].set_xlim(0, 78)

    for n, v in enumerate(df_plot['skill_percent']):
        ax[i].text(v + 1, n, f'{v:.0f}%', va='center')

    if i != len(job_titles) - 1:
        ax[i].set_xticks([])

fig.suptitle('Likelihood of Skills Requested in US Job Postings', fontsize=15)
plt.tight_layout(h_pad=0.5) # firx the overlap
plt.show()